# LightBox API - Purpose of Condo Logic

Use an address to acquire its location coordinates, then use those coordinates to find an associated property parcel, then use that parcel to find associated tax assessment data.

# LightBox API - Using Addresses to Find Tax Assessment Data

This notebook demonstrates basic features of the LightBox Geocoding API by walking through the following steps:

1. Import Python packages
2. Enter your **API Key** for authorization
3. Create request objects and display the results both in the JSON form.

Additional Materials:
[LightBox Developer Portal](https://developer.lightboxre.com/)

#1. Import the required python packages

In [30]:
import requests
import json
from typing import Dict
import pandas as pd


#2. Import function definitions

In [31]:
# ----------------------------
# Function Definitions
# ----------------------------

# Function to geocode a single address using the LightBox API.
def geocode_address(lightbox_api_key: str, address: str) -> Dict:
    """
    Geocodes the provided address using the LightBox API.
    
    Args:
        lightbox_api_key (str): The API key for accessing the LightBox API.
        address (str): The address string for matching.
    
    Returns:
        dict: The geocoded address information in JSON format.
    """
    # API endpoint configuration
    BASE_URL = "https://api.lightboxre.com/v1"
    ENDPOINT = "/addresses/search"
    URL = BASE_URL + ENDPOINT

    # Setting up request parameters and headers
    params = {"text": address}
    headers = {"x-api-key": lightbox_api_key}

    # Sending request to the LightBox API
    geocoder_data = requests.get(URL, params=params, headers=headers)
    
    # Returning the geocoded address information
    return geocoder_data


# Test for the get_parcel_from_lbx_address_id() function
def get_parcel_data_from_address_coordinates(lightbox_api_key: str, country_code: str, address_wkt_coordinates: str) -> Dict:
    """
    Returns a dictionary containing the parcel data for the specified address.

    Args:
        lightbox_api_key (str): The API key for accessing the LightBox API.
        country_code (str): The country code for the address.
        address_wkt_coordinates (str): The address coordinates for the address.
    """

    # API endpoint configuration
    BASE_URL = "https://api.lightboxre.com/v1"
    ENDPOINT = f"/parcels/{country_code}/geometry"
    URL = BASE_URL + ENDPOINT

    # Setting up request parameters and headers
    params = {"wkt": address_wkt_coordinates}
    headers = {"x-api-key": lightbox_api_key}

    # Make the request
    parcel_data = requests.get(URL, params=params, headers=headers)
    
    # Return the parcel data
    return parcel_data


def get_assessment_data_from_lbx_parcel_id(lightbox_api_key: str, parcel_id: str) -> Dict:
    """
    Returns a dictionary containing the assessment data for the specified parcel.

    Args:
        lightbox_api_key (str): The LightBox API key.
        parcel_id (str): The parcel ID.
    """

    # API endpoint configuration
    BASE_URL = "https://api.lightboxre.com/v1"
    ENDPOINT = f"/assessments/_on/parcel/us/{parcel_id}"
    URL = BASE_URL + ENDPOINT

    # Setting up request parameters and headers
    headers = {"x-api-key": lightbox_api_key}

    # Make the request
    assessment_data = requests.get(URL, headers=headers)
    
    # Return the assessment data
    return assessment_data


#3. Create variables that will be used to authenticate your calls.
Get your key from the [LightBox Developer Portal](https://developer.lightboxre.com/).

In [39]:
lightbox_api_key = '<YOUR_API_KEY>'

#4. LightBox API Details
This notbook will create various requests and display the output in JSON.
1. **Geocoding** - https://api.lightboxre.com/v1/addresses/search
2. **Parcels** - https://api.lightboxre.com/v1/parcels/{country_code}/geometry
3. **Assessments** - https://api.lightboxre.com/v1/assessments/_on/parcel/us/{lbx_parcel_id}

For additional details regarding each endpoint's request parameters or response models, visit the [API Catalog](https://developer.lightboxre.com/apis) on the LightBox Developer Portal page.

Create variables containing necessary inputs for subsequent calls.


1. Notice how the API returns any addresses within the buffer distance of the provided geometry, as well as any metadata associated with that address

In [73]:
# ----------------------------
# API Usage
# ----------------------------

# Specify the address to geocode
address = "24299 Paseo De Valencia, Laguna Woods, CA 92637"
country_code = "us" # 'us' for the United States

# Geocode the specified address
address_search_data = geocode_address(lightbox_api_key, address)

# Get the parcel data from the geocoded address
parcel_data = get_parcel_data_from_address_coordinates(lightbox_api_key, country_code, address_search_data.json()["addresses"][0]["location"]["representativePoint"]["geometry"]["wkt"])

# Get the assessment data from the parcel ID
assessment_data = get_assessment_data_from_lbx_parcel_id(lightbox_api_key, parcel_data.json()["parcels"][0]["id"])


In [74]:
pd.json_normalize(address_search_data.json()["addresses"])

,$ref,id,uuaid,uaid,label,type,addressConfidenceScore,postalCodeDerived,corroborationIndex,parcels,...,location.locality,location.regionCode,location.unit,location.county,location.countryCode,location.postalCode,location.postalCodeExt,location.representativePoint.latitude,location.representativePoint.longitude,location.representativePoint.geometry.wkt
0,https://api.lightboxre.com/v1/addresses/search...,06000B911R4H3YWETLFGHQ,None,None,"24299 PASEO DE VALENCIA, LAGUNA WOODS, CA 9263...",None,None,None,None,[{'$ref': 'https://api.lightboxre.com/v1/parce...,...,LAGUNA WOODS,CA,None,ORANGE COUNTY,US,92637,3100,33.609982,-117.713722,POINT(-117.713722 33.609982)


In [75]:
pd.json_normalize(parcel_data.json()["parcels"])


,$ref,id,fips,parcelApn,county,legalDescription,neighborhood,opportunityZone,subdivision,topography,...,transaction.lastMarketSale.tdDocumentNumber,transaction.lastMarketSale.deedTransactionType,transaction.lastMarketSale.lenderType,transaction.priorMarketSale.transferDate,transaction.priorMarketSale.lender,transaction.multipleApnFlag,usplss.township,usplss.range,usplss.section,usplss.quarter
0,https://api.lightboxre.com/v1/parcels/us/0202P...,0202P9GBKVNO0LIUTDCJ47,06059,621-191-16,ORANGE,[N TR 16835 BLK LOT 1 UN HOLD],None,False,None,None,...,None,None,None,None,,None,None,None,None,None


In [76]:
pd.json_normalize(assessment_data.json()["assessments"])


,$ref,apn,fips,id,$metadata,alternateApn,avm,improvementPercent,assessedLotSize,poolIndicator,...,tax.amount,tax.rateAreaCode,tax.exemption,valuationModel.value,valuationModel.valueHigh,valuationModel.valueLow,valuationModel.date,valuationModel.propensityScore,valuationModel.score,valuationModel.label
0,https://api.lightboxre.com/v1/assessments/us/0...,621-191-16,06059,03046JQUZ4Y7Q7TATH24S3,None,None,NaN,NaN,12353.31191,None,...,NaN,32-010,None,NaN,NaN,NaN,None,None,NaN,None
1,https://api.lightboxre.com/v1/assessments/us/0...,932-810-75,06059,03018O7YHXQ8WN8ENIA2R1,None,None,615166.0,49.90,NaN,None,...,6139.0,32-010,None,692000.0,816560.0,567440.0,None,None,82.0,$692 K
2,https://api.lightboxre.com/v1/assessments/us/0...,932-810-76,06059,030759YM7ZE8HWFQCF0TTH,None,None,459833.0,58.04,NaN,None,...,4099.0,32-010,None,555000.0,627150.0,482850.0,None,None,87.0,$555 K
3,https://api.lightboxre.com/v1/assessments/us/0...,932-810-77,06059,0303YGY9HW6C14HQCMJFC7,None,None,599500.0,51.57,NaN,None,...,5385.0,32-010,None,588000.0,699720.0,476280.0,None,None,81.0,$588 K
4,https://api.lightboxre.com/v1/assessments/us/0...,932-810-78,06059,0304XNNLW2WT4VF74Q75N4,None,None,599500.0,51.57,NaN,None,...,5385.0,32-010,None,588000.0,699720.0,476280.0,None,None,81.0,$588 K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,https://api.lightboxre.com/v1/assessments/us/0...,932-812-04,06059,03002OKN1NJDVLT53YPMRJ,None,None,566500.0,52.95,NaN,None,...,5408.0,32-010,None,590000.0,708000.0,472000.0,None,None,80.0,$590 K
131,https://api.lightboxre.com/v1/assessments/us/0...,932-812-05,06059,0306DPN3Y95BZLE3G29MIC,None,None,566500.0,52.95,NaN,None,...,5408.0,32-010,None,590000.0,708000.0,472000.0,None,None,80.0,$590 K
132,https://api.lightboxre.com/v1/assessments/us/0...,932-812-06,06059,03060SKL4MQ53HIX3X1AJT,None,None,373833.0,66.49,NaN,None,...,3117.0,32-010,None,391000.0,469200.0,312800.0,None,None,80.0,$391 K
133,https://api.lightboxre.com/v1/assessments/us/0...,932-812-07,06059,0306OVUEJWYCVIZ6WC8HLA,None,None,615166.0,49.90,NaN,None,...,6139.0,32-010,None,692000.0,816560.0,567440.0,None,None,82.0,$692 K


In [78]:
# --------------------
# Print collected data
# --------------------

# Print the geocoded address data in a readable JSON format
print(json.dumps(address_search_data.json(), indent=4))
# Print the parcel data in a readable JSON format
print(json.dumps(parcel_data.json(), indent=4))
# Print the assessment data in a readable JSON format
print(json.dumps(assessment_data.json(), indent=4))



{
    "$ref": "https://api.lightboxre.com/v1/addresses/search?text=24299%20Paseo%20De%20Valencia%2C%20Laguna%20Woods%2C%20CA%2092637",
    "$metadata": {
        "geogcs": {
            "epsg": "4326"
        },
        "recordSet": {
            "totalRecords": 1.0
        }
    },
    "addresses": [
        {
            "$ref": "https://api.lightboxre.com/v1/addresses/search?text=24299%20Paseo%20de%20Valencia%2C%20Laguna%20Woods%2C%20CA%2092637-3100%2C%20USA",
            "id": "06000B911R4H3YWETLFGHQ",
            "uuaid": null,
            "uaid": null,
            "$metadata": {
                "geocode": {
                    "confidence": {
                        "streetNumber": 1,
                        "streetName": 1,
                        "locality": 1,
                        "postalCode": 1,
                        "score": 1
                    },
                    "precisionCode": 10,
                    "addressComponents": {
                        "prefixType":

# Commonly Seen Errors

See [LightBox Geocoder API](https://developer.lightboxre.com/api/geocoding#/Autocomplete/get_addresses__autocomplete) for a list of common error responses. 